In [2]:
# Save as: generate_bpo_data.py
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from faker import Faker
import random
import json

fake = Faker()
np.random.seed(42)

# Configuration
NUM_AGENTS = 500
RECORDS_PER_AGENT = 4   
ATTRITION_RATE = 0.22

# Generate Agents (with messiness)
agents_data = []
for i in range(NUM_AGENTS):
    agent_id = f"AG{i:05d}"
    
    # 5% missing first names
    first_name = None if random.random() < 0.05 else fake.first_name()
    
    # Inconsistent shift casing
    shift_options = ['DAY', 'day', 'Day', 'NIGHT', 'night', 'Night', 'MID', 'mid']
    
    # 2% future hire dates (data entry error)
    hire_date = fake.date_between(start_date='-3y', end_date='today')
    if random.random() < 0.02:
        hire_date = fake.date_between(start_date='today', end_date='+30d')
    
    agents_data.append({
        'agent_id': agent_id,
        'first_name': first_name,
        'last_name': fake.last_name(),
        'email': fake.email() if random.random() > 0.03 else None,
        'hire_date': hire_date,
        'shift_type': random.choice(shift_options),
        'campaign': random.choice(['Sales', 'Support', 'Retention', 'Collections', 'Tech Support']),
        'site_location': random.choice(['Manila', 'Quezon City', 'Cebu', 'Davao', 'Makati']),
        'employment_status': random.choice(['Active', 'Probation', 'LOA', 'Suspended']),
        'team_lead': f"TL{random.randint(1,20):03d}"
    })

# Add 5 duplicate agent records
for _ in range(5):
    dup_idx = random.randint(0, len(agents_data)-1)
    agents_data.append(agents_data[dup_idx].copy())

agents_df = pd.DataFrame(agents_data)

# Generate Performance Logs (with messiness)
perf_data = []
log_id_counter = 1

for agent in agents_df['agent_id'].unique()[:NUM_AGENTS]:  # Exclude duplicates for logs
    for _ in range(RECORDS_PER_AGENT):
        log_date = fake.date_between(start_date='-6m', end_date='today')
        
        # Date format variations
        date_formats = [
            log_date.strftime('%Y-%m-%d'),
            log_date.strftime('%m/%d/%Y'),
            log_date.strftime('%d-%b-%y')
        ]
        formatted_date = random.choice(date_formats)
        
        # Realistic BPO metrics with outliers
        calls = random.randint(30, 120) if random.random() > 0.02 else random.randint(-10, 0)  # 2% negative
        aht = random.randint(200, 600) if random.random() > 0.03 else random.randint(2000, 5000)  # 3% outliers
        fcr = round(random.uniform(0.65, 0.95), 2) if random.random() > 0.10 else None  # 10% null
        csat = random.randint(1, 5) if random.random() > 0.05 else random.choice([0, 6, 7, None])  # 5% out of range
        quality = round(random.uniform(75, 98), 1) if random.random() > 0.08 else None
        adherence = round(random.uniform(85, 100), 1)
        occupancy = round(random.uniform(75, 92), 1)
        aux_time = random.randint(30, 90)
        
        perf_data.append({
            'log_id': f"LOG{log_id_counter:06d}",
            'agent_id': agent,
            'log_date': formatted_date,
            'calls_handled': calls,
            'aht_seconds': aht,
            'fcr_rate': fcr,
            'csat_score': csat,
            'quality_score': quality,
            'adherence_pct': adherence,
            'occupancy_pct': occupancy,
            'aux_time_min': aux_time,
            'notes': fake.sentence() if random.random() > 0.60 else None
        })
        log_id_counter += 1

# Add 10 complete duplicate logs
for _ in range(10):
    perf_data.append(random.choice(perf_data).copy())

perf_df = pd.DataFrame(perf_data)

# Generate Attrition Records
attrition_data = []
attrited_agents = np.random.choice(
    agents_df['agent_id'].unique()[:NUM_AGENTS], 
    size=int(NUM_AGENTS * ATTRITION_RATE), 
    replace=False
)

for agent in attrited_agents:
    # 3% future attrition dates
    if random.random() < 0.03:
        attr_date = fake.date_between(start_date='today', end_date='+60d')
    else:
        attr_date = fake.date_between(start_date='-12m', end_date='today')
    
    # Inconsistent boolean formats
    bool_formats = ['Yes', 'Y', '1', 'TRUE', 'true']
    bool_formats_no = ['No', 'N', '0', 'FALSE', 'false']
    
    voluntary = random.choice([True, False])
    
    attrition_data.append({
        'agent_id': agent,
        'attrition_date': attr_date,
        'attrition_flag': 1,
        'reason_code': random.choice(['Resignation', 'NCNS', 'Performance', 'Relocation', None]) if random.random() > 0.15 else None,
        'voluntary_flag': random.choice(bool_formats) if voluntary else random.choice(bool_formats_no),
        'notice_period_days': random.randint(0, 30) if voluntary else 0,
        'exit_interview_completed': random.choice(bool_formats) if random.random() > 0.30 else random.choice(bool_formats_no),
        'rehire_eligible': random.choice(['Eligible', 'Not Eligible', 'Under Review', None])
    })

# Add non-attrited agents
for agent in agents_df['agent_id'].unique()[:NUM_AGENTS]:
    if agent not in attrited_agents:
        attrition_data.append({
            'agent_id': agent,
            'attrition_date': None,
            'attrition_flag': 0,
            'reason_code': None,
            'voluntary_flag': 'N',
            'notice_period_days': None,
            'exit_interview_completed': None,
            'rehire_eligible': 'Eligible'
        })

attrition_df = pd.DataFrame(attrition_data)

# Generate Schedule JSON (with messiness)
schedule_data = []
for agent in np.random.choice(agents_df['agent_id'].unique()[:NUM_AGENTS], size=100):
    for _ in range(random.randint(1, 3)):
        sched_date = fake.date_between(start_date='-30d', end_date='today')
        
        # Timezone mixing (UTC vs Asia/Manila)
        if random.random() < 0.3:
            scheduled_start = f"{sched_date}T08:00:00Z"  # UTC
            scheduled_end = f"{sched_date}T17:00:00Z"
        else:
            scheduled_start = f"{sched_date}T08:00:00+08:00"  # PHT
            scheduled_end = f"{sched_date}T17:00:00+08:00"
        
        # Late punch-ins
        actual_start = scheduled_start.replace('08:', f"{random.randint(8,9):02d}:")
        late_flag = random.random() < 0.15
        
        schedule_data.append({
            'agent_id': agent,
            'schedule_date': str(sched_date),
            'scheduled_start': scheduled_start,
            'scheduled_end': scheduled_end,
            'actual_start': actual_start,
            'actual_end': scheduled_end,
            'break_duration': random.randint(30, 60),
            'late_flag': late_flag
        })

# Save files
agents_df.to_csv('agents.csv', index=False)
perf_df.to_csv('performance_logs.csv', index=False)
attrition_df.to_csv('attrition_records.csv', index=False)

# Introduce JSON formatting errors (5% of records)
with open('schedule_data.json', 'w') as f:
    json_str = json.dumps(schedule_data, indent=2)
    # Remove random commas to create errors
    if random.random() < 0.05:
        json_str = json_str.replace(',\n', '\n', random.randint(1, 3))
    f.write(json_str)

print("Data generation complete!")
print(f"Agents: {len(agents_df)} records")
print(f"Performance Logs: {len(perf_df)} records")
print(f"Attrition: {len(attrition_df)} records")
print(f"Schedule: {len(schedule_data)} records")

Data generation complete!
Agents: 505 records
Performance Logs: 2010 records
Attrition: 500 records
Schedule: 196 records
